### Import Libraries

In [1]:
from netgen.geom2d import *
import scipy
import scipy.sparse.linalg
from ngsolve import *
from ngsolve.webgui import Draw
import numpy as np
from scipy.linalg import block_diag

### Set Parameters

In [2]:
a = 2              # Length of rectangle in x direction 
b = 1             # Length of rectangle in y direction 

maxh = 0.1   # Mesh-size
order = 1      # Order of Polynomials
nr_eigs = 100     # Number of Eigenvalues to approximate

### Create the Rectangle

In [3]:
geo = SplineGeometry()
geo.AddRectangle((0,0), (a,b), bcs=["b","r","t","l"])
mesh = Mesh(geo.GenerateMesh(maxh=maxh))
Draw(mesh)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008', 'mesh_dim': 2, 'order2d': 1, 'order3d': 1, 'draw_vol': N…

### finite element space and bilinear form

In [4]:
fes = H1(mesh, complex=True, order=order)

eigenvec = GridFunction(fes,multidim=nr_eigs)

u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm (fes)
a += SymbolicBFI (grad(u)*grad(v))

b = BilinearForm (fes)
b += SymbolicBFI (u*v)

a.Assemble()
b.Assemble()

A=a.mat
B=b.mat

print(fes.ndof)


259


### eigenvalue solver via scipy

In [5]:
shift = 2

shifted = a.mat.CreateMatrix()
shifted.AsVector().data = a.mat.AsVector() - (shift**2)*b.mat.AsVector()
invshifted = shifted.Inverse(freedofs=fes.FreeDofs())
print(a.mat.height)
tmp1 = eigenvec.vec.CreateVector()
tmp2 = eigenvec.vec.CreateVector()
def matvec(v):
    tmp1.FV().NumPy()[:] = v
    tmp2.data = b.mat * tmp1
    tmp1.data = invshifted * tmp2
    return tmp1.FV().NumPy()

A = scipy.sparse.linalg.LinearOperator( (a.mat.height,a.mat.width), matvec)
mu, vecs = scipy.sparse.linalg.eigs(A,nr_eigs)

lamsquared=shift**2+1/mu
lams=np.sort([sqrt(l) for l in lamsquared])
print("resonances:")
for i in range (0,len(lams)):
    print(i,": ",lams[i], " squared: ", lamsquared[i])  


259
resonances:
0 :  1.5769906706002895e-07  squared:  (2.4710368035387607+4.1710890978802304e-15j)
1 :  1.5719531810899332  squared:  (2.4868995751603507e-14+1.904680167114359e-14j)
2 :  3.150899793509489  squared:  (9.92816950873814+3.983655361082426e-14j)
3 :  3.152133369056001  squared:  (9.935944776316335+1.1949192073587066e-15j)
4 :  3.5274631817040563  squared:  (12.442996498277704-3.561601875498378e-15j)
5 :  4.472809429474393  squared:  (20.006024192395046-7.517688541997756e-15j)
6 :  4.743851478232357  squared:  (22.50412684752732+2.4908377959219602e-14j)
7 :  5.723504427182917  squared:  (32.75850292798245-2.5666176498120816e-14j)
8 :  6.358188626202136  squared:  (40.426562606366204-1.0834620881510832e-14j)
9 :  6.368684822001105  squared:  (40.56014636198725+1.7173605207464027e-14j)
10 :  6.5701047405527255  squared:  (43.16627630183339-2.361509089549511e-14j)
11 :  7.137139167860725  squared:  (51.033593215790276-2.766699058577776e-14j)
12 :  7.14378003691255  squared:  (

C:\Users\Salli\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: ComplexWarning: Casting complex values to real discards the imaginary part


### copy and plot eigenfunctions

In [6]:
for i in range(nr_eigs):
    eigenvec.vecs[i].FV().NumPy()[:] = vecs[:,i]

Draw(eigenvec)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

In [7]:
def QR_simple(A,tol = 1e-6):
    count = 0
    while abs(A[1,0]) > tol:
        Q,R = np.linalg.qr(A)
        A = R@Q
        count +=1
    return A, sorted(np.diag(A)) ,count



def QR_shift(A,tol=1e-6):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol:
            rho = A[i,i]
            Q,R = np.linalg.qr(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count +=1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count



def QR_shift2(A,tol=1e-6):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol*(abs(A[i-1,i-1])+abs(A[i,i])):
            w = np.linalg.eigvals(A[i-1:i+1,i-1:i+1])
            if abs(w[0] - A[i,i]) < abs(w[1] - A[i,i]):
                rho = w[0]
            else:
                rho = w[1]
            Q,R = np.linalg.qr(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count += 1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count



def QR_hesse(A,tol=1e-6):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol*(abs(A[i-1,i-1])+abs(A[i,i])):
            w = np.linalg.eigvals(A[i-1:i+1,i-1:i+1])
            if abs(w[0] - A[i,i]) < abs(w[1] - A[i,i]):
                rho = w[0]
            else:
                rho = w[1]
            Q,R = QR_decomp_hesse(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count += 1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A))[::-1], count



def QR_decomp_hesse(A):
    n = A.shape[0]
    Q = np.eye(n,n)

    for i in range(0,n-1):
        if abs(A[i,i]) >= abs(A[i+1,i]):
            t = A[i+1,i]/abs(A[i,i])
            root = (1+abs(t)**2)**(1/2)
            c = A[i,i]/(abs(A[i,i])*root)
            s = t/root
        else:
            t = A[i,i]/abs(A[i+1,i])
            root = (1+abs(t)**2)**(1/2)
            s = A[i+1,i]/(abs(A[i+1,i])*root)
            c = t/root
            
        M = np.array([[c.conj(), s.conj()], [-s, c]])
        G = block_diag(np.eye(i,i), M, np.eye(n-i-2, n-i-2))
        Q = G@Q
        for j in range(i, n):
            temp_1 = A[i,j]
            A[i,j] = c.conj()*temp_1 + s.conj()*A[i+1,j]
            A[i+1,j] = -s*temp_1 + c*A[i+1,j]
    return Q.T.conj(), A



def lanczos(A,k = 0):
    n = A.shape[1]
    if k == 0:
        k = n
        
    v0 = np.random.rand(n)
    v = [v0/np.linalg.norm(v0)]
    gam = [v[0].T.conj()@A@v[0]]
    w = (A - gam[0]*np.identity(n))@v[0]
    delta = [np.linalg.norm(w)]
    i = 0
    while delta[i] > 1e-12 and i<k-1:
        v.append(w/delta[i])
        i +=1
        gam.append(v[i].T.conj()@A@v[i])
        w = (A - gam[i]*np.identity(n))@v[i] - delta[i-1]*v[i-1]
        delta.append(np.linalg.norm(w))
    T = np.diag(delta[:-1], -1) + np.diag(gam) + np.diag(delta[:-1], 1)
    return QR_hesse(T)



def arnoldi(A, dim, k = 0):
    n = dim
    if k == 0:
        k = n
        
    v0 = np.random.rand(n)
    v = [v0/np.linalg.norm(v0)]
    h = np.zeros((k,k))
    
    for j in range(k):
        w = A(v[j])
        for l in range(j+1):
            h[l][j] = v[l].T.conj()@w
            w = w - h[l][j]*v[l]
        if j < k-1:
            h[j+1][j] = np.sqrt(w.T.conj()@w)
            if abs(h[j+1][j]) < 1e-14:
                return QR_hesse(h[:j+1,:j+1])
            else:
                v.append(w/h[j+1][j])
    
    return QR_hesse(h)

In [9]:
mu = arnoldi(matvec, a.mat.height)[1]

for i in range(len(mu)):
    mu[i] = shift**2+1/mu[i]
lamsquared_1 = mu
lams_1=[sqrt(l) for l in lamsquared_1]
print("resonances:")
for i in range (0,len(lams_1)):
    print(i,": ",lams_1[i], " squared: ", lamsquared_1[i])  


C:\Users\Salli\Anaconda3\lib\site-packages\ipykernel_launcher.py:122: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\Salli\Anaconda3\lib\site-packages\ipykernel_launcher.py:125: ComplexWarning: Casting complex values to real discards the imaginary part


resonances:
0 :  3.150899793509483  squared:  9.928169508738103
1 :  3.152133369056002  squared:  9.93594477631634
2 :  3.5274631817040527  squared:  12.442996498277678
3 :  4.472809429474378  squared:  20.00602419239491
4 :  4.743851478232334  squared:  22.504126847527104
5 :  5.723504427182909  squared:  32.758502927982356
6 :  6.358188626206385  squared:  40.426562606420234
7 :  6.368684822024311  squared:  40.56014636228283
8 :  6.570104740522059  squared:  43.166276301430436
9 :  7.137139167860743  squared:  50.93875550141195
10 :  7.143780036912578  squared:  51.033593215790674
11 :  8.000777947564375  squared:  64.01244776623241
12 :  8.020305719202057  squared:  64.32530382946521
13 :  8.651600798002486  squared:  74.85019636799724
14 :  9.123032249292589  squared:  83.22971742163259
15 :  9.679036351146793  squared:  93.68374468682104
16 :  9.716176317071195  squared:  94.40408222441518
17 :  9.861074248582879  squared:  97.24078533606438
18 :  10.24155804951617  squared:  104

C:\Users\Salli\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.
